In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os,csv,math,sys, joblib
import numpy as np
import sklearn.model_selection, sklearn.preprocessing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
class neural_network(nn.Module):
    def __init__(self, num_input, num_hidden,num_output):
        super(neural_network, self).__init__()
        self.hidden = nn.Linear(num_input,num_hidden).cuda()
        self.out = nn.Linear(num_hidden,num_output).cuda()

    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.out(x)

In [3]:
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
dataset = pd.read_csv('Iris_Dataset.csv')
dataset = pd.get_dummies(dataset, columns=['Species']) # One Hot Encoding
values = list(dataset.columns.values)

In [5]:
y = dataset[values[-3:]]
y = np.array(y, dtype='float32')
X = dataset[values[:-3]]
X = np.array(X, dtype='float32')

In [9]:
# Shuffle Data
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 30
X_test = X_values[-test_size:]
X_train = X_values[:-test_size]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

joblib.dump(scaler, 'scaler.pkl') 
y_test = y_values[-test_size:]


y_train = y_values[:-test_size]

# Interval / Epochs
interval = 50
epoch = 500

loss_func = nn.CrossEntropyLoss(weight=None, size_average=True, ignore_index=-100, reduce=True)

# Input neurons : 4
# Hidden neurons : 8
# Output neurons : 3
hidden_layer_nodes = 2
model = neural_network(4,hidden_layer_nodes,3)

weights, biases = [], []
for name, p in model.named_parameters():
    if 'bias' in name:
        biases += [p]
    else:
        weights += [p]
optimizer = optim.Adam([
                {'params': weights},
                {'params': biases}
                ], lr = 0.0003, weight_decay=1e-5)

for epoch in range(epoch): 
    loss_val = 0
    for input_data in enumerate(X_train):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        input_neurons_data = autograd.Variable(torch.cuda.FloatTensor(input_data[1]))

        one_hot_target = y_train[input_data[0]]
        target_class_index = int(np.where(one_hot_target == 1.0)[0])

        target_class = autograd.Variable(torch.cuda.LongTensor([target_class_index]))
        output = model(input_neurons_data)
        
        loss = loss_func(output.view(1,-1),target_class)
        loss_val += loss

        loss.backward()
        optimizer.step()
    if epoch%interval == 0:
        print loss_val.data[0]/len(X_train)

#torch.save(model, "iris_model_2")
torch.save(model, "iris_model_" + str(hidden_layer_nodes))

error = 0.0
for i in range(len(X_test)):
    predicted = np.argmax(np.array(model(autograd.Variable(torch.cuda.FloatTensor(X_test[i]))).data))
    actual = int(np.where(y_test[i] == 1.0)[0])
    if predicted != actual:
        print predicted, actual
        error += 1.0

print error/len(X_test)

error = 0.0
for i in range(len(X_train)):
    predicted = np.argmax(np.array(model(autograd.Variable(torch.cuda.FloatTensor(X_train[i]))).data))
    actual = int(np.where(y_train[i] == 1.0)[0])
    if predicted != actual:
        print predicted, actual
        error += 1.0

print error/len(X_train)

/home/cs18mtech11019/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:65: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(1.1974, device='cuda:0')
tensor(0.7720, device='cuda:0')
tensor(0.4063, device='cuda:0')
tensor(0.2687, device='cuda:0')
tensor(0.1959, device='cuda:0')
tensor(0.1504, device='cuda:0')
tensor(0.1210, device='cuda:0')
tensor(0.1014, device='cuda:0')
tensor(0.0877, device='cuda:0')
tensor(0.0776, device='cuda:0')
2 1
0.0333333333333
1 2
2 1
2 1
2 1
0.0333333333333


/home/cs18mtech11019/anaconda2/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type neural_network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
